In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [6]:
df = pd.read_csv(r'F:\user_to_1st_time_investor.csv')

In [7]:
df.head()

,guid,investor_status,distinct_days_visited_website,distinct_properties_viewed,time_on_properties,interactions_on_property_page,times_marketplace_filter_used,dashboard_interactions,time_on_howitworks,time_on_faqs,time_on_aboutus,time_on_team,time_on_homepage,time_on_marketplace,time_to_sign_up_from_first_visit_days,first_visit_to_first_investment_days
0,prod-549e3913-6a2f-4de8-91e3-10a5713abf19,investor,3,5,1175,17,0,0,68,25,117,0,195,605,0,86
1,prod-84f4f3d4-623a-4087-8c8c-924f503e40c3,investor,10,14,8738,16,0,0,131,640,0,536,620,3688,0,36
2,prod-354808e9-6c56-4ecc-acef-8fdac8685cd3,investor,4,7,771,5,0,0,567,0,0,129,93,45,0,29
3,prod-b03ffe67-d0d8-416a-94d0-5feff6f0430c,investor,5,4,178,2,0,0,202,0,0,0,44,55,0,0
4,prod-3af51e10-b3d3-4b8e-b89c-2f6b4b8130fb,investor,14,15,6614,40,0,0,46,1327,0,0,224,674,4,4


In [8]:
df.columns

Index(['guid', 'investor_status', 'distinct_days_visited_website',
       'distinct_properties_viewed', 'time_on_properties',
       'interactions_on_property_page', 'times_marketplace_filter_used',
       'dashboard_interactions', 'time_on_howitworks', 'time_on_faqs',
       'time_on_aboutus', 'time_on_team', 'time_on_homepage',
       'time_on_marketplace', 'time_to_sign_up_from_first_visit_days',
       'first_visit_to_first_investment_days'],
      dtype='object')

In [9]:
df_drop = df[[
'time_on_homepage',
'time_on_aboutus',
'time_on_howitworks',
'distinct_days_visited_website',
'investor_status']]

In [10]:
df_drop.head()

,time_on_homepage,time_on_aboutus,time_on_howitworks,distinct_days_visited_website,investor_status
0,195,117,68,3,investor
1,620,0,131,10,investor
2,93,0,567,4,investor
3,44,0,202,5,investor
4,224,0,46,14,investor


In [11]:
df_drop.investor_status.value_counts()

non-investor    262142
investor          1359
Name: investor_status, dtype: int64

In [12]:
df_drop.investor_status[df_drop.investor_status == 'non-investor'] = 0
df_drop.investor_status[df_drop.investor_status == 'investor'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C

In [13]:
df_drop.investor_status.value_counts()

0    262142
1      1359
Name: investor_status, dtype: int64

#Undersampling
#Cause non_investor >> investor
#Select number of non_investor == investor

In [14]:
df_non_investor = df_drop[df_drop.investor_status == 0].sample(n=1359, random_state=200)

In [15]:
df_investor = df_drop[df_drop.investor_status == 1]

In [16]:
df_new = pd.concat([df_investor, df_non_investor])

In [17]:
df_new.describe()

,time_on_homepage,time_on_aboutus,time_on_howitworks,distinct_days_visited_website
count,2718.000000,2718.000000,2718.000000,2718.000000
mean,166.564753,5.706034,92.325975,4.965416
std,474.673879,32.201016,186.221447,9.016297
min,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,1.000000
50%,43.000000,0.000000,0.000000,2.000000
75%,187.000000,0.000000,115.000000,6.000000
max,14475.000000,542.000000,3346.000000,263.000000


In [18]:
df_new.columns

Index(['time_on_homepage', 'time_on_aboutus', 'time_on_howitworks',
       'distinct_days_visited_website', 'investor_status'],
      dtype='object')

#RandomForestClassifier

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [20]:
train=df_new.sample(frac=0.8,random_state=200)
test=df_new.drop(train.index)

In [21]:
clf = RandomForestClassifier()
clf.fit(train.drop('investor_status', axis=1), list(train.investor_status.values))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [22]:
predicted = clf.predict(test.drop('investor_status', axis=1))

In [23]:
print(clf.feature_importances_)

[ 0.4248449   0.00778326  0.07315255  0.49421929]


In [24]:
df_new.columns

Index(['time_on_homepage', 'time_on_aboutus', 'time_on_howitworks',
       'distinct_days_visited_website', 'investor_status'],
      dtype='object')

In [25]:
from sklearn import metrics

In [26]:
print(metrics.classification_report(list(test.investor_status.values), predicted))

             precision    recall  f1-score   support

          0       0.93      0.94      0.93       275
          1       0.94      0.93      0.93       269

avg / total       0.93      0.93      0.93       544



#Data is not undersampling df2

In [27]:
df2 = df[['time_on_homepage', 'time_on_aboutus', 'time_on_howitworks',
       'distinct_days_visited_website','investor_status']]

In [28]:
df2.investor_status[df2.investor_status == 'non-investor'] = 0
df2.investor_status[df2.investor_status == 'investor'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C

In [29]:
predicted_new = clf.predict(df2.drop('investor_status', axis=1))

In [30]:
print(metrics.classification_report(list(df2.investor_status.values), predicted_new))

             precision    recall  f1-score   support

          0       1.00      0.92      0.96    262142
          1       0.06      0.97      0.12      1359

avg / total       0.99      0.92      0.96    263501



#PCA reduce the dimension

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

In [32]:
pca = PCA(n_components=0.99, whiten=True)

In [33]:
X_pca = pca.fit_transform(df_new)

In [34]:
print(df_new.shape[1])
print(X_pca.shape[1])

5
2


#advice: use knn